In [ ]:
def build_generator():
    input_shape = (256, 256, 3)  # Input image shape
    num_filters = 64  # Number of filters in the first layer
    num_downsampling = 2  # Number of downsampling layers
    num_residual_blocks = 9  # Number of residual blocks
    num_upsampling = 2  # Number of upsampling layers
    output_channels = 3  # Number of output channels (RGB)

    # Input layer
    inputs = keras.Input(shape=input_shape)

    # Downsampling
    x = layers.Conv2D(num_filters, 7, padding="same")(inputs)
    x = layers.Activation("relu")(x)

    for _ in range(num_downsampling):
        # Convolutional layers with downsampling
        num_filters *= 2
        x = layers.Conv2D(num_filters, 3, strides=2, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, num_filters)

    # Upsampling
    for _ in range(num_upsampling):
        # Convolutional layers with upsampling
        num_filters //= 2
        x = layers.Conv2DTranspose(num_filters, 3, strides=2, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

    # Output layer
    x = layers.Conv2D(output_channels, 7, padding="same")(x)
    outputs = layers.Activation("tanh")(x)

    # Build the generator model
    generator = keras.Model(inputs, outputs, name="generator")
    return generator

def residual_block(x, num_filters):
    # Residual block with skip connections
    y = x

    # First convolutional layer
    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    # Second convolutional layer
    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Skip connection
    x = layers.Add()([x, y])
    x = layers.Activation("relu")(x)
    return x

In [ ]:
def build_discriminator():
    input_shape = (256, 256, 3)  # Input image shape
    num_filters = 64  # Number of filters in the first layer
    num_downsampling = 3  # Number of downsampling layers
    output_channels = 1  # Number of output channels (binary classification)

    # Input layer
    inputs = keras.Input(shape=input_shape)

    # Convolutional layers
    x = layers.Conv2D(num_filters, 4, strides=2, padding="same")(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)

    for i in range(1, num_downsampling):
        # Convolutional layers with downsampling
        num_filters *= 2
        x = layers.Conv2D(num_filters, 4, strides=2, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.LeakyReLU(alpha=0.2)(x)

    # Output layer
    x = layers.Conv2D(output_channels, 4, strides=1, padding="same")(x)
    outputs = layers.Activation("sigmoid")(x)

    # Build the discriminator model
    discriminator = keras.Model(inputs, outputs, name="discriminator")
    return discriminator

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
    
def generator_loss(fake_output):
    gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    return gen_loss

In [ ]:
# Build the generators
generator_AB = build_generator()  # Real to cartoon
generator_BA = build_generator()  # Cartoon to real

# Build the discriminators
discriminator_A = build_discriminator()  # Real discriminator
discriminator_B = build_discriminator()  # Cartoon discriminator

# Define the loss functions (binary cross-entropy)
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
# Optimizers for generators and discriminators
generator_optimizer = keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)

In [ ]:
def train_step(real_images, cartoon_images):
    with tf.GradientTape(persistent=True) as tape:
        # Generate fake images
        fake_cartoons = generator_AB(real_images)
        fake_real = generator_BA(cartoon_images)

        # Calculate cycle consistency loss
        cycle_real_images = generator_BA(fake_cartoons)
        cycle_cartoon_images = generator_AB(fake_real)
        cycle_loss = tf.reduce_mean(tf.abs(real_images - cycle_real_images)) + tf.reduce_mean(tf.abs(cartoon_images - cycle_cartoon_images))

        # Calculate total generator loss
        gen_loss = generator_loss(fake_output)

        # Calculate discriminator losses
        disc_loss_A = discriminator_loss(real_output, fake_output)
        disc_loss_B = discriminator_loss(real_output, fake_output)

    # Calculate the gradients for generators and discriminators
    generator_gradients = tape.gradient(gen_loss, generator_AB.trainable_variables)
    discriminator_gradients_A = tape.gradient(disc_loss_A, discriminator_A.trainable_variables)
    discriminator_gradients_B = tape.gradient(disc_loss_B, discriminator_B.trainable_variables)

    # Update the weights of the generators and discriminators
    generator_optimizer.apply_gradients(zip(generator_gradients, generator_AB.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients_A, discriminator_A.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients_B, discriminator_B.trainable_variables))

# Build the generators
generator_AB = build_generator()  # Real to cartoon
generator_BA = build_generator()  # Cartoon to real

# Build the discriminators
discriminator_A = build_discriminator()  # Real discriminator
discriminator_B = build_discriminator()  # Cartoon discriminator

# Define the loss functions (binary cross-entropy)
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

# Training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        for real_images, cartoon_images in dataset:
            train_step(real_images, cartoon_images)



# first methode

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import random_rotation, random_zoom

def preprocess_image(image_path, target_size):
    # Load the image and resize
    img = load_img(image_path, target_size=target_size)
    # Convert the image to array
    img = img_to_array(img)
    # Normalize the pixel values to the range of [-1, 1]
    img = (img - 127.5) / 127.5
    return img

def preprocess_dataset(image_paths, target_size):
    images = []
    for image_path in image_paths:
        # Preprocess each image
        img = preprocess_image(image_path, target_size)
        images.append(img)
    return np.array(images)

def augment_dataset(images, batch_size):
    augmented_images = []
    for image in images:
        # Random rotation
        image = random_rotation(image, 30, row_axis=0, col_axis=1, channel_axis=2)
        # Random zoom
        image = random_zoom(image, (0.8, 1.2), row_axis=0, col_axis=1, channel_axis=2)
        augmented_images.append(image)
    return np.array(augmented_images)

# Example usage
real_image_paths = ['path/to/real_image1.jpg', 'path/to/real_image2.jpg', ...]
cartoon_image_paths = ['path/to/cartoon_image1.jpg', 'path/to/cartoon_image2.jpg', ...]

target_size = (256, 256)  # Desired image size

# Preprocess the real images
real_images = preprocess_dataset(real_image_paths, target_size)

# Preprocess the cartoon images
cartoon_images = preprocess_dataset(cartoon_image_paths, target_size)

# Augment the datasets
real_images_augmented = augment_dataset(real_images, batch_size)
cartoon_images_augmented = augment_dataset(cartoon_images, batch_size)



In [ ]:

epochs = 100
batch_size = 16

# Create your dataset using augmented real and cartoon images
dataset = tf.data.Dataset.from_tensor_slices((real_images_augmented, cartoon_images_augmented))
dataset = dataset.shuffle(buffer_size=num_samples * 2).batch(batch_size)

# Training loop
for epoch in range(epochs):
    for real_images, cartoon_images in dataset:
        # Perform training step
        train_step(real_images, cartoon_images)

# deusieme methode

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths to your real and cartoon image directories
real_image_dir = 'path/to/real_images'
cartoon_image_dir = 'path/to/cartoon_images'

# Set the target size for resizing the images
target_size = (256, 256)

# Set the batch size for training
batch_size = 16

# Create data generators for real and cartoon images
data_generator = ImageDataGenerator(preprocessing_function=lambda x: (x - 127.5) / 127.5)

real_data_generator = data_generator.flow_from_directory(
    real_image_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=True
)

cartoon_data_generator = data_generator.flow_from_directory(
    cartoon_image_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=True
)

# Combine the real and cartoon data generators into a single dataset generator
dataset_generator = zip(real_data_generator, cartoon_data_generator)